# LeetCode #7. Reverse Integer (Medium)

---

**Prompt:** Given a signed 32-bit integer x, return x with its digits reversed. If reversing x causes the value to go outside the signed 32-bit integer range [-2\**31, 2\**31 - 1], then return 0.

Assume the environment does not allow you to store 64-bit integers (signed or unsigned).

**Example 1:** Output: 321

In [5]:
x1 = 123

**Example 2:** Output: -321

In [4]:
x2 = -123

**Example 3:** Output: 21

In [6]:
x3 = 120

The first approach I chose was quite simple and focused on the mathematical implementation.
1. To isolate last digit in base b, perform modulus (remainder division) -- **x%b**
2. To remove last digit in base b, perform integer division -- **x//10**
3. To increment place of numbers, multiple by b -- **x\*b**
4. Must store sign from beginning because at the end of the integer traversal, it is true that -0 = +0

I took into account the 32-bit integer range constraint with the following conditional equation: (a)
    
    INT_MIN <= rev_x * 10 + x_last_digit <= INT_MAX

and manipulated it into the following conditionals: (b)

    (INT_MIN - x_last_digit) <= rev_x * 10 <= (INT_MAX - x_last_digit)
    
and manipulated further to isolate rev_x, the value to return: (c)

    (INT_MIN - x_last_digit) / 10 <= rev_x <= (INT_MAX - x_last_digit) / 10

Initially, I was hesitant to use biconditional statement (c) because I did not want to account for partial division and the creation of a decimal (float). For this reason, I chose to base my conditionals on biconditional statement (b)

Here is my first solution, which had a runtime of 29 ms and used 13.8MB of memory.

In [8]:
class Solution:
    def reverse(self, x: int) -> int:

        INT_MIN = -(2**31)
        INT_MAX = 2**31 - 1

        rev_x = 0

        if x < 0:
            sign = -1
            x = -x
        else:
            sign = 1
        
        while x != 0:
            
            x_last_digit = x % 10

            min_rev_x = INT_MIN - x_last_digit
            max_rev_x = INT_MAX - x_last_digit

            if rev_x*10 >= min_rev_x and rev_x*10 <= max_rev_x:
                rev_x = rev_x * 10 + x_last_digit
            else:
                return 0

            x = x // 10

        return rev_x * sign

While LeetCode accepted my Python solution, Flo realized that my solution violated the 32-bit range constraint when she used C++ which had the 32-bit compiling constraint. Python for this LeetCode problem was more forgiving.

After reasoning through the problem and simulating a simpler version (8-bit range constraint where x = 128), we realized a few things:
1. The bit integer violation was occurring when rev_x\*10 was greater than INT_MAX. We determined that biconditional statement (c) was accurate when integer division was used and resulted in 3 cases, most notably the boundary condition case where rev_x == max_rev_x and the last digit of x to be added to rev_x matters because that digit could cause rev_x to be greater than INT_MAX
2. Because I store the sign of x and then force x to be positive to perform the modulus calculation in Python, I do not need a double-sided comparison and effectively the range of values is from 0 to INT_MAX
3. The last digit of INT_MAX is one less than the last digit of INT_MIN

Here is my *final solution*, which stays within the bounds of the bit integer range constraint for all calculations completed. The runtime for this solution is 35ms and uses 13.8MB of memory.

In [9]:
class Solution:
    def reverse(self, x: int) -> int:

        # Completed on 4/17/2023

        # Defining Constants
        INT_MAX = 2**31 - 1
        INT_MAX_LAST_DIGIT = INT_MAX % 10
        INT_MAX_REV_X = INT_MAX//10
        INT_MIN = -(2**31)
        INT_MIN_LAST_DIGIT = INT_MAX_LAST_DIGIT + 1

        rev_x = 0

        # python does not handle modulus of a negative number the way I want, so making x positive and 
        if x < 0:
            sign = -1
            # x = -x would not work when x is equal to INT_MIN
            # because -INT_MIN is +2**31, which is INT_MAX + 1
            # I will create base case to account for this limitation
            if x == INT_MIN:
                return 0
            else:
                x = -x
        else:
            sign = 1
        
        while x != 0:
            
            x_last_digit = x % 10
            
            # rev_x*10 would ALMOST exceed INT_MAX, we need to look at the last digit
            if rev_x == INT_MAX_REV_X:
                # x was originally negative
                if sign == -1 and x_last_digit <= INT_MIN_LAST_DIGIT:
                    rev_x = rev_x * 10 + x_last_digit
                # x was originally positive
                elif sign == 1 and x_last_digit <= INT_MAX_LAST_DIGIT:
                    rev_x = rev_x * 10 + x_last_digit
                # adding x_last_digit would push us past INT_MIN or INT_MAX
                else:
                    return 0
            # rev_x*10 would exceed INT_MAX
            elif rev_x > INT_MAX_REV_X:
                return 0
            # it is safe to add the last digit
            else:
                rev_x = rev_x * 10 + x_last_digit

            x = x // 10

        return rev_x * sign

In [11]:
self.reverse(x1)

NameError: name 'self' is not defined